# MLflow results presentation

In [108]:
import pandas  as pd

In [93]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [94]:
import mlflow
mlflow.set_tracking_uri('file:///home/zeyusun/work/flare-prediction-smarp/mlruns')

from mlflow.tracking import MlflowClient
client = MlflowClient()

In [103]:
exp_id = client.get_experiment_by_name("experiment").experiment_id
runs = mlflow.search_runs(exp_id)
#runs = mlflow.search_runs(“<experiment_id>”, “metrics.loss < 2.5”)
#runs[['params.num_epochs','metrics.loss']].groupby("params.num_epochs").agg("min")

In [104]:
run_id = runs.loc[runs['tags.mlflow.runName'] == 'beta', 'run_id'].item()
tags = [
    'tags.database_name',
    'tags.dataset_name',
    'tags.estimator_name',
]
params = [
]
metrics = ['metrics.' + m for m in [
    'auc',
    'tss',
    'tss_opt',
    'hss2',
    'precision',
    'recall',
]]
df = runs.loc[(runs['tags.mlflow.parentRunId'] == run_id) &
              (runs['tags.estimator_name'] != 'HistGradientBoostingClassifier'),
              tags + params + metrics]
df_copy = df = df.rename(columns=lambda k: k.split('.')[1])
df = df.set_index(['database_name', 'dataset_name', 'estimator_name']).unstack(-1).T
df

database_name                    MX_Q_12hr                     MX_Q_12hr_balanced  ...  MX_Q_6hr MX_Q_6hr_balanced                    
dataset_name                      combined     sharp     smarp           combined  ...     smarp          combined     sharp     smarp
          estimator_name                                                           ...                                                
auc       RandomForestClassifier  1.000000  1.000000  1.000000           1.000000  ...  1.000000          1.000000  1.000000  1.000000
          SGDClassifier           0.992902  0.992745  0.992889           0.992424  ...  0.992576          0.993801  0.993878  0.993757
tss       RandomForestClassifier  1.000000  1.000000  1.000000           1.000000  ...  1.000000          1.000000  1.000000  1.000000
          SGDClassifier           0.916268  0.917968  0.914547           0.916385  ...  0.922587          0.919825  0.925656  0.922741
tss_opt   RandomForestClassifier  1.000000  1.000000  1.000000           1.000000  ...  1.000000          1.000000  1.000000  1.000000
          SGDClassifier           0.920719  0.922775  0.916903           0.917230  ...  0.927015          0.931487  0.927114  0.927114
hss2      RandomForestClassifier  1.000000  1.000000  1.000000           1.000000  ...  1.000000          1.000000  1.000000  1.000000
          SGDClassifier           0.451291  0.456210  0.467752           0.916385  ...  0.313382          0.919825  0.925656  0.922741
precision RandomForestClassifier  1.000000  1.000000  1.000000           1.000000  ...  1.000000          1.000000  1.000000  1.000000
          SGDClassifier           0.310326  0.314356  0.324757           0.955500  ...  0.196929          0.948791  0.964861  0.959361
recall    RandomForestClassifier  1.000000  1.000000  1.000000           1.000000  ...  1.000000          1.000000  1.000000  1.000000
          SGDClassifier           0.964527  0.965372  0.959459           0.961149  ...  0.972303          0.972303  0.960641  0.963557

[12 rows x 18 columns]

In [89]:
df_style = df.style.background_gradient(axis=None)#, vmin=0.7)
df_style

In [40]:
print(df.round(3).to_latex())

\begin{tabular}{llrrrrrrrrrrrrrrrrrr}
\toprule
       & database\_name & \multicolumn{3}{l}{MX\_Q\_12hr} & \multicolumn{3}{l}{MX\_Q\_12hr\_balanced} & \multicolumn{3}{l}{MX\_Q\_24hr} & \multicolumn{3}{l}{MX\_Q\_24hr\_balanced} & \multicolumn{3}{l}{MX\_Q\_6hr} & \multicolumn{3}{l}{MX\_Q\_6hr\_balanced} \\
       & dataset\_name &  combined &  sharp &  smarp &           combined &  sharp &  smarp &  combined &  sharp &  smarp &           combined &  sharp &  smarp & combined &  sharp &  smarp &          combined &  sharp &  smarp \\
{} & estimator\_name &           &        &        &                    &        &        &           &        &        &                    &        &        &          &        &        &                   &        &        \\
\midrule
auc & HistGradientBoostingClassifier &     1.000 &  0.994 &  0.994 &              1.000 &  0.999 &  0.994 &     1.000 &  0.992 &  0.993 &              1.000 &  0.993 &  0.996 &    1.000 &  0.994 &  0.994 &             1.000 &

## Deep learning methods

In [99]:
exp_id = client.get_experiment_by_name("experiment_arnet").experiment_id
runs = mlflow.search_runs(exp_id)
parentRunId = runs.loc[runs['tags.mlflow.runName'] == 'alpha', 'run_id'].item()
runs = runs[runs['tags.mlflow.parentRunId'] == parentRunId]

In [101]:
#run_id = runs.loc[runs['tags.mlflow.runName'] == 'arnet', 'run_id'].item()
tags = [
    #'tags.mlflow.runName',
    'tags.database_name',
    'tags.dataset_name',
    'tags.estimator_name',]
params = [
]
metrics = ['metrics.test/' + m for m in [
    'auc',
    'tss',
    'hss2',
    'precision',
    'recall',
]]
df2 = runs.loc[:,
            #(runs['tags.estimator_name'] != 'HistGradientBoostingClassifier'),
            tags + params + metrics]
df2 = df2.rename(columns=lambda k: k.split('.')[-1])
df2_copy = df2 = df2.rename(columns=lambda k: k.split('/')[-1] if '/' in k else k)
df2 = df2.set_index(['database_name', 'dataset_name', 'estimator_name']).unstack(-1).T
df2

database_name            NaN MX_Q_6hr_balanced                    
dataset_name             NaN          combined     sharp     smarp
          estimator_name                                          
auc       NaN            NaN               NaN       NaN       NaN
          C3D            NaN               NaN  0.726221       NaN
          LSTM           NaN          0.979295  0.965406  0.950709
          MLP            NaN          0.963112  0.961414  0.947860
tss       NaN            NaN               NaN       NaN       NaN
          C3D            NaN               NaN  0.414747       NaN
          LSTM           NaN          0.847926  0.824885  0.773504
          MLP            NaN          0.811060  0.788018  0.756410
hss2      NaN            NaN               NaN       NaN       NaN
          C3D            NaN               NaN  0.414747       NaN
          LSTM           NaN          0.847926  0.824885  0.773504
          MLP            NaN          0.811060  0.788018  0.756410
precision NaN            NaN               NaN       NaN       NaN
          C3D            NaN               NaN  0.736842       NaN
          LSTM           NaN          0.910714  0.874477  0.895196
          MLP            NaN          0.866667  0.876652  0.844358
recall    NaN            NaN               NaN       NaN       NaN
          C3D            NaN               NaN  0.645161       NaN
          LSTM           NaN          0.940092  0.963134  0.876068
          MLP            NaN          0.958525  0.917051  0.927350

In [112]:
df3 = pd.concat((df_copy, df2_copy.iloc[1:]))
df3 = df3.set_index(['database_name', 'dataset_name', 'estimator_name']).unstack(-1).T
df3.style.background_gradient(axis=None)#, vmin=0.7)

In [114]:
import numpy as np
from sklearn.model_selection import LeavePOut

X = np.ones(4)
lpo = LeavePOut(p=2)
for train, test in lpo.split(X):
    print("%s %s" % (train, test))

[2 3] [0 1]
[1 3] [0 2]
[1 2] [0 3]
[0 3] [1 2]
[0 2] [1 3]
[0 1] [2 3]


In [122]:
from pathlib import Path
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [123]:
from run_sklearn import get_dataset
X_train, X_test, y_train, y_test, groups_train, _ = get_dataset(Path('../datasets/preprocessed/M_QS_6hr/'), 'smarp')

usage: ipykernel_launcher.py [-h] [-d DATA_ROOT] [-s] [-e EXPERIMENT_NAME] [-r RUN_NAME] [-o OUTPUT_DIR]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/zeyusun/.local/share/jupyter/runtime/kernel-1b156af3-f98d-405e-8f68-79c3ad1b60cb.json
[autoreload of run_sklearn failed: Traceback (most recent call last):
  File "/home/zeyusun/.conda/envs/arnet/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/zeyusun/.conda/envs/arnet/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/zeyusun/.conda/envs/arnet/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/zeyusun/.conda/envs/arnet/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_exter

NameError: name 'cfg' is not defined